In [0]:
edgesDF = (
  sqlContext.read
    .format('com.databricks.spark.csv')
    .options(delimiter=',', header='true', inferschema='true')
    .load("/FileStore/tables/comp4651-project/musae_squirrel_edges.csv")
)

trafficDF = (
    sqlContext.read
    .format('com.databricks.spark.csv')
    .options(delimiter=',', header='true', inferschema='true')
    .load("/FileStore/tables/comp4651-project/musae_squirrel_target.csv")
)

In [0]:
num_nodes = trafficDF.count()
num_edges = edgesDF.count()
print("num of nodes: {}".format(num_nodes))
print("num of edges: {}".format(num_edges))

num of nodes: 5201
num of edges: 217073

In [0]:
### PART 1.1 - Analysis the distribution of node degree ###
# The graph is undirected, deg of a node equals the sum of count in id1 and id2

nodeDeg = (
  edgesDF.select('id1')
          .union(edgesDF.select('id2'))
          .groupBy('id1')
          .count()
          .toDF('id', 'degree')
)

In [0]:
from pyspark.sql.functions import expr
# determine the distribution function of node degree

nodeDegStat = (
  nodeDeg.groupBy('degree').count()
    .withColumn('logD', expr('log10(degree)')) # degree on log scale
    .withColumn('pdf', expr('count/{}'.format(num_nodes))) # probalistic distribution function (pdf)
    .withColumn('logp', expr('log10(pdf)')) # pdf on log scale
    .withColumn('cdf', expr('sum(pdf) over (order by degree)')) # cummulative distribution function (cdf)
    .withColumn('ccdf', expr('1-cdf')) # comptemporary cummulative distribution fucntion (ccdf)
    .withColumn('log1-P', expr('log10(ccdf)')) # ccdf on log scale
)

In [0]:
# drop the last row for display purpose
display(nodeDegStat.head(nodeDegStat.count()-1))

degree,count,logD,pdf,logp,cdf,ccdf,log1-P
1,137,0.0,0.02634108825225918,-1.5793662866184253,0.02634108825225918,0.9736589117477408,-0.011593156765533455
2,157,0.3010299956639812,0.030186502595654682,-1.5201872013655984,0.05652759084791387,0.9434724091520861,-0.02527079579391666
3,182,0.47712125471966244,0.034993270524899055,-1.4560154657897573,0.09152086137281293,0.9084791386271871,-0.04168504092955036
4,192,0.6020599913279624,0.03691597769659681,-1.4327856250712825,0.12843683906940973,0.8715631609305903,-0.05970113471614432
5,202,0.6989700043360189,0.03883868486829456,-1.4107354843282083,0.16727552393770428,0.8327244760622957,-0.0794986700449898
6,186,0.7781512503836436,0.03576235339357816,-1.446573909556916,0.20303787733128245,0.7969621226687176,-0.09856231888853975
7,225,0.8450980400142568,0.043260911363199386,-1.3639043356634697,0.24629878869448185,0.7537012113055181,-0.12280078675437489
8,171,0.9030899869919435,0.03287829263603153,-1.4830907433826783,0.2791770813305134,0.7208229186694866,-0.14217141335328148
9,160,0.9542425094393249,0.030763314747164008,-1.5119668711189074,0.30994039607767737,0.6900596039223226,-0.1611133954415923
10,147,1.0,0.02826379542395693,-1.548769519026656,0.3382041915016343,0.6617958084983657,-0.17927598778329065


In [0]:
# perform linear regrssion on logD against logp (pdf)

# perform linear regression on logD against log1-P (ccdf)

In [0]:
### PART 1.2 - analysis the distribution of node traffic ###

trafficStat = (trafficDF
                 .groupBy(expr('cast(log10(target) as decimal(5, 1)) as logT')) # group the traffic data in log scale, round down to 1 decimal space
                 .count()
                 .orderBy('logT')
                 .withColumn('pdf', expr('count/{}'.format(num_nodes))) # probalistic distribution function
                 .withColumn('cdf', expr('sum(pdf) over (order by logT)')) # cummulative distribution function
              )

In [0]:
display(trafficStat)

logT,count,pdf,cdf
1.2,1,1.9227071716977504E-4,1.9227071716977504E-4
1.6,3,5.768121515093252E-4,7.690828686791001E-4
1.7,6,0.0011536243030186503,0.0019227071716977505
1.8,14,0.0026917900403768506,0.004614497212074601
1.9,20,0.003845414343395501,0.008459911555470103
2.0,34,0.006537204383772351,0.014997115939242453
2.1,35,0.006729475100942127,0.02172659104018458
2.2,23,0.004422226494904826,0.026148817535089408
2.3,41,0.007883099403960777,0.03403191693905019
2.4,44,0.008459911555470103,0.04249182849452029


In [0]:
logTraffic = trafficDF.selectExpr('log10(target) as logT')
print(logTraffic.selectExpr('mean(logT) as mean').first())
print(logTraffic.selectExpr('percentile_approx(logT, 0.5) as median').first())
print(trafficDF.selectExpr('log10(target) as logT').selectExpr('stddev(logT) as sd').first())

Row(mean=3.8877600669518904)
Row(median=3.9649192942866427)
Row(sd=0.766662369755757)

In [0]:
### PART 1.3 - analysis the relationship between the node degree and traffic size ###

nodeDF = nodeDeg.join(trafficDF, nodeDeg.id == trafficDF.id)
display(nodeDF)

id,degree,id,target
4935,428,4935,3173
4101,10,4101,29367
1959,120,1959,21075
1829,17,1829,66
3749,421,3749,487
2659,5,2659,25639
1088,3,1088,9410
3918,11,3918,434
148,184,148,222045
1645,32,1645,43592
